In [1]:
import pandas as pd
import os
import math
from scipy.optimize import fsolve



In [2]:
#functions to save data frame
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
"""Functions to Calculate Contact Stress for Roller"""
def calcualteRollerContactStress(W,*args):
    """Parameter Description 
    rtype=Types of Roller 1:Pneumatic 2:Smooth Wheel 3:Grid Roller 4:Sheepfoot Roller 5:Vibratory Roller
    W=Weight of rollers in Matric Ton
    d=diameter of roller drum
    b=length of roller drum
    c=constants 0.175 for on rigid soil 0.275 for soft soil
    tp="Tire Pressue for Pneumatic Roller"
    I =dynamic factor
    cr= coverage ratio 0.5 for Grid Roller, 0.8-0.12 for sheepfoot roller 1 for all other
    A=Contact Area as per Grecenko(1995) 
    Wd=driving wheel weight =0.66w
    """
    rtype=args[0]
    d=args[1]
    b=args[2]
    tp=args[3]
    cr=args[4]
    I=args[5]
    if rtype==1:
        sigma=tp
    else: 
        c=0.225
        A=c*d*b
        Wd=0.66*W
        sigma=(Wd*I)/(A*cr)
    sigma=float(math.floor(sigma))
    return sigma
def designMaxLiftThickness(B,*args):
    """Parameter Description    
    F=percent of soil particle passing #200 sieve in decimal
    gamma=Unit weight of Soil in mton/cum
    sigma=contact stress in mton/sqm    
    """   
    F=args[0]
    gamma=args[1]
    sigma=args[2]
    term1=0.25*F*math.pow(gamma,2.5)
    term2=(1.0/6.0)*B*gamma
    term3=math.exp(3.8*gamma-1)
    print("F={} gamma={} sigma={}".format(F,gamma,sigma))
    print("term1={} term2={} term3={}".format(term1,term2,term3))
    return (term1+term2)*term3-sigma
def designMinimumNoOfRollerPasses(gamma_initial,gamma_final,F,sigma):
    t1=17.5*(1-0.5*F)
    t2=t1/sigma
    t3=math.pow(gamma_final,7)-math.pow(gamma_initial,7)
    no_passes=math.ceil(t2*t3)
    return no_passes
def buildRollerSpec(rtype,weight):
    """Parameter Description""" 

    roller_class=["Pneumatic","Smooth Wheeled",
                  "Grid Roller","Sheepfoot Roller","Vibratory Roller"]
    roller_spec=str(weight)+" Ton "+roller_class[rtype-1]
    return  roller_spec
def calculateHeff2(B,F,gamma,sigmaNot):
    term1=gamma*B+1.5*F*math.pow(gamma,2.5)
    term2=8*sigmaNot
    term3=math.log(term1/term2)
    effective_height=1.4*B*(1-3.8*gamma-term3)
    effective_height=round(effective_height,2)
    if effective_height< B:
        effective_height=B
    return  effective_height
def compactionDetails(*args):
    totalpasses=args[0]
    F=args[1]
    gamma_old=args[2]
    Ld=args[3] #lift thickness
    CP=args[4] #Contact Pressure
    pass_no=[]
    print("Lift Thickness={}".format(Ld))
    gamma_initial_list=[]
    Initial_Lift_thickness_list=[]
    Elastic_modulus_list=[]
    Effective_Depth_list=[]
    lift_settlement_list=[]
    Final_Lift_thickness=[]
    Final_gamma_list=[]
    Hn=Ld
    for i in range(1, totalpasses+1):
        print("passno={}".format(i))
        es=calcualteEs(F,gamma_old)
        Heff= calculateHeff2(Ld,F,gamma_old,CP)
        #Heff=Hn
        settlement=calcualteLiftSettlement(Heff,Ld,CP,es)
        print("settle_ment={}".format(settlement))
        Hn1=Hn-settlement
        gamma_new=round((Hn/Hn1)*gamma_old,2)
        """Transfering Data to Lists"""
        pass_no.append(i)
        gamma_initial_list.append(gamma_old)
        Initial_Lift_thickness_list.append(Hn*100)
        Elastic_modulus_list.append(es)
        Effective_Depth_list.append(Heff)
        lift_settlement_list.append(settlement)
        Final_Lift_thickness.append(Hn1*100)
        Final_gamma_list.append(gamma_new)
        Hn=Hn1
        gamma_old=gamma_new
    mydata={"Pass_No":pass_no,"Initial_Dry_Density":gamma_initial_list,
            "Initial-Lift_Thickness":Initial_Lift_thickness_list,
           "Elastic_Modulus":Elastic_modulus_list,"Heff": Effective_Depth_list,
            "Settlement": lift_settlement_list,"Final-Lift_Thickness":Final_Lift_thickness,
            "Final_Dry_Density":Final_gamma_list
           }
        
        
        
    myframe=pd.DataFrame(data= mydata)
    return myframe

    
    



In [4]:
"""Functions to Data Transfer"""



'Functions to Data Transfer'

In [5]:
"""Soil Proprertie Calculator"""
def CalculateUltimateBearingCapacity(gamma_target,F):
    B=0.15
    term1=(F*math.pow(gamma_target,2.5))/4.0
    term2=(gamma_target*B)/6.0
    term3=math.exp(3.8*gamma_target-1)
    sigma_ult=(term1+term2)*term3
    return sigma_ult
    
    
def calcualteEs(F,gamma):
    elasticity=30.0*(1-0.5*F)*math.pow(gamma,9.0)
    elasticity=float(math.ceil(elasticity))
    return elasticity
def calculateHeff(B,F,gamma,sigmaNot):
    term1=gamma*B+1.5*F*math.pow(gamma,2.5)
    term2=8*sigmaNot
    term3=math.log(term1/term2)
    effective_height=1.4*B*(1-3.8*gamma-term3)
    effective_height=round(effective_height,2)    
    return  effective_height

def calcualteLiftSettlement(Heff,B,sigmaNot,Es):
    print("Heff={} B={}".format(Heff,B))
    A=0.555*math.log(Heff/B)+0.93
    delh=(A*B*sigmaNot)/Es
    return delh
def calcualteContactWidth(gamma_target,Roller_Diameter):
    B=(0.45-0.15*gamma_target)*Roller_Diameter
    B=math.floor(B*100)
    return B
def calculateRequiredMinimumSigma(F,gamma_target,B):
    term1=(F*math.pow(gamma_target,2.5))/4.0
    term2=(gamma_target*B)/6.0
    term3=math.exp(3.8*gamma_target-1)
    sigma_min=(term1+term2)*term3
    return sigma_min
def calculateMaximumLiftThicknessFromSigma(sigma,F,gamma_target):
    term1=math.exp(3.8*gamma_target-1)
    term2=sigma/term1
    term3=(F*math.pow(gamma_target,2.5))/4.0
    term4=term2-term3
    B=(term4*6)/gamma_target
    return B
    
def calcualteMinimumComaptorWeight(F,gamma_target,Roller_diameter):
    B=calcualteContactWidth(gamma_target,Roller_diameter)
    B=B/100.0
    term1=(F*math.pow(gamma_target,2.5))/4.0
    term2=(gamma_target*B)/6.0
    term3=math.exp(3.8*gamma_target-1)
    sigma_min=(term1+term2)*term3
    I=1.0
    S=0.5
    minimum_compactor_weight=(sigma_min*S)/(1.2*I)
    minimum_compactor_weight=math.ceil(minimum_compactor_weight)
    sigma_min2=math.floor(sigma_min)
    myvalue={"mcw":minimum_compactor_weight,"sigma":sigma_min2}
    return myvalue
def calcualteRollerPasses(gamma_initial,gamma_final,F,Roller_diameter):
    B=calcualteContactWidth(gamma_final,Roller_diameter)
    myvalue=calcualteMinimumComaptorWeight(F,gamma_final,Roller_diameter)
    sigma=myvalue["sigma"]
    t1=17.5*(1-0.5*F)
    t2=t1/sigma
    t3=math.pow(gamma_final,7)-math.pow(gamma_initial,7)
    no_passes=math.ceil(t2*t3)
    myreturn_val={"LT":B,"CWT":myvalue["mcw"],"Sigma":myvalue["sigma"],"passes":no_passes}    
    return myreturn_val
def calcualteRollerPassesForFixedLiftThickness(gamma_initial,gamma_final,F,Roller_Weight,Roller_Dia,Lift_thicness):
    #B=calcualteContactWidth(gamma_final,Roller_diameter)
    B=Lift_thicness/100.0  
    B_Max=calcualteContactWidth(gamma_final,Roller_Dia)/100.00    
    if B>B_Max:
        B=B_Max      
    #myvalue=calcualteMinimumComaptorWeight(F,gamma_final,Roller_diameter)
    I=1.0
    S=0.5
    sigma=(Roller_Weight*1.2*I)/S
    sigma_min=calculateRequiredMinimumSigma(F,gamma_final,B)
    if sigma_min>sigma:
        B_revised=calculateMaximumLiftThicknessFromSigma(sigma,F,gamma_final)
        print("Lift Thicknes Revision necessary revised LT={} cm".format(B_revised*100))
        
    t1=17.5*(1-0.5*F)
    t2=t1/sigma
    t3=math.pow(gamma_final,7)-math.pow(gamma_initial,7)
    no_passes=math.ceil(t2*t3)
    myreturn_val={"LT":B,"Sigma":sigma,"passes":no_passes}    
    return myreturn_val

def calcualteTotalContactPressureForSmoothedWheeLRollers(W,*args):
    """Parameter Description 
    W=Weight of rollers in Matric Ton
    d=diameter of roller drum
    b=length of roller drum
    c=constants 0.175 for on rigid soil 0.275 for soft soil
    A=Contact Area as per Grecenko(1995) 
    Wd=driving wheel weight =0.66w
    """    
    d=args[0]
    b=args[1]
    dynamic_factor=args[2]
    c=0.225
    A=c*d*b
    Wd=0.66*W
    sigma=float(math.floor(dynamic_factor*Wd/A))
    return sigma
    
    

    

In [6]:
myfolder=r'F:\website\cmis6\Civilworks cost\Soil Compaction' # office computer
input_path=os.path.join(myfolder,"Input.xlsx")
output_path=os.path.join(myfolder,"Output.xlsx")
sheetName="Parameter"
parameter_df=pd.read_excel(input_path,sheet_name=sheetName)
parameter_df.fillna("",inplace=True)
parameter_df
"""Design Adequcy of Roller"""
roller_type=int(parameter_df.iloc[0,3])
roller_weight=parameter_df.iloc[1,3]
drum_dia=parameter_df.iloc[2,3]
drum_length=parameter_df.iloc[3,3]
tp=parameter_df.iloc[4,3]
coverage_ratio=parameter_df.iloc[5,3]
dynamic_factor=parameter_df.iloc[6,3]
gammanot=parameter_df.iloc[7,3]
gammafinal=parameter_df.iloc[8,3]
F=parameter_df.iloc[9,3]
Lmin=parameter_df.iloc[10,3]
Lmax=parameter_df.iloc[11,3]
roller_adequete=False
"""Checking Roller Adequecy"""
CP=calcualteRollerContactStress(roller_weight,roller_type,drum_dia,drum_length,tp,coverage_ratio,dynamic_factor)
UB=CalculateUltimateBearingCapacity(gammafinal,F)
if CP>UB:
    roller_adequete=True
print("sigma={} UB={} roller_adequete={}".format(CP,UB,roller_adequete))
Ld=fsolve(designMaxLiftThickness,Lmax/100,args=(F,gammafinal,CP))
Ld_cm=(math.floor(Ld*100))
#print("design Lift thickness={}cm".format(Ld_cm))
passes=designMinimumNoOfRollerPasses(gammanot,gammafinal,F,CP)
print("design Lift thickness={}cm no passes={}".format(Ld_cm,passes))
roller_spec=buildRollerSpec(roller_type,roller_weight)
sheetName="Output_summary"
design_summary_df=pd.read_excel(input_path,sheet_name=sheetName)

mylist=[roller_spec,math.ceil(UB),math.floor(CP),Ld_cm,passes]
design_summary_df=design_summary_df.assign(value=mylist)
"""Calcualting Compactions Details"""
sheetName="Output_Details"
compaction_details_df=pd.read_excel(input_path,sheet_name=sheetName)
compaction_details=compactionDetails(passes,F,gammanot,Ld_cm/100,math.floor(CP))




myframes=[]
mynames=[]
myframes.append(parameter_df)
mynames.append("Input Parameter")
myframes.append(design_summary_df)
mynames.append("Design Summary")
myframes.append(compaction_details)
mynames.append("Design Details")
saveDataFrame(myframes,output_path,mynames)






sigma=183.0 UB=176.7351628527572 roller_adequete=True
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.0875] term3=284.2914658239207
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.0875] term3=284.2914658239207
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.0875] term3=284.2914658239207
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.0875] term3=284.2914658239207
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.06578667] term3=284.2914658239207
F=0.5706 gamma=1.75 sigma=183.0
term1=0.5779189000514009 term2=[0.06578667] term3=284.2914658239207
design Lift thickness=22cm no passes=3
Lift Thickness=0.22
passno=1
Heff=0.66 B=0.22
settle_ment=0.1399312021708507
passno=2
Heff=0.22 B=0.22
settle_ment=9.396476089326478e-06
passno=3
Heff=0.22 B=0.22
settle_ment=9.396476089326478e-06


In [7]:
parameter_df

,Name,Symbol,unit,value,Explanation
0,Roller Type,,,5.0000,1:Pneumatic 2:Smooth Wheel 3:Grid Roller 4:She...
1,Roller Weight,W,ton,30.0000,
2,drum dia,d,meter,1.2000,
3,drum length,b,meter,1.0000,
4,tire pressure,,ton/sqm,0.0000,
5,covearge ratio,,,1.0000,"0.5 for Grid Roller, 0.8-0.12 for sheepfoot ro..."
6,Dynamic Factor,I,,2.5000,2.0-2.50 for roller type 5 1 for all others
7,gammanot,,ton/cum,1.4000,initial dry density of loose fill
8,gammafinal,,ton/cum,1.7500,final dry density of embankment Soil
9,fine_percent,F,percent,0.5706,in decimal e.g. 10% for 0.1


In [8]:
input_path
"""Extracting Design Parameter"""
gammaInitial=parameter_df.iloc[0,2]
liftThicknessInitial=parameter_df.iloc[1,2]
sigmaNot=parameter_df.iloc[2,2]
fine_percent=parameter_df.iloc[3,2]
contact_width=parameter_df.iloc[4,2]
Hn=liftThicknessInitial
gamma_old=gammaInitial
for i in range(1,9):
    print("gamma_ini={} Hi={} sigmaNot={}".format(gamma_old,Hn,sigmaNot))
es=calcualteEs(fine_percent,gammaInitial)
Heff= calculateHeff(contact_width,fine_percent,gammaInitial,sigmaNot)
settlement=calcualteLiftSettlement(Heff,contact_width,sigmaNot,es)

Hn1=Hn-settlement
gamma_new=round((Hn/Hn1)*gamma_old,2)
print("Youn's Modulus={} Heff={} settlement={} gamma_new={}".format(es,Heff,settlement,gamma_new))
#F,gamma_target,B
myval=calcualteMinimumComaptorWeight(0.10,1.75,1.2)
B=calcualteContactWidth(1.75,1.2)
print("Comapctor Weight={} metric_ton compaction_stress={} psi lift_thickness={} cm".format(myval["mcw"],myval["sigma"],B))
myvalue=calcualteRollerPasses(1.70,1.75,0.50,1.2)
print("Roller={}mton sigma={} lift={} cm passe={} nos".format(myvalue["CWT"],myvalue["Sigma"],myvalue["LT"],myvalue["passes"]))
myvalue=calcualteRollerPassesForFixedLiftThickness(1.39,1.75,0.66,20,1.2,22)
print("sigma={} Thickness={} no of passes={}".format(myvalue["Sigma"],myvalue["LT"],myvalue["passes"]))

gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter
gamma_ini= Hi=ton sigmaNot=meter


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
math.log(10)

In [ ]:
B=0.00001
design_sigma=designMaxLiftThickness(B,0,1.7,0)
cp=calcualteTotalContactPressureForSmoothedWheeLRollers(25,1.0,1.2,1)
print("minimum contact pressure={} ton/sqm".format(design_sigma))

In [ ]:
rval=fsolve(designMaxLiftThickness,0.1,args=(0.87,1.7,200))
max_lift_thickness=math.floor(rval[0]*100)
print("maximum lift thickness={} cm".format(max_lift_thickness))
cp=calcualteTotalContactPressureForSmoothedWheeLRollers(25,1.0,1.2,2)
print("minimum contact pressure={} ton/sqm".format(cp))